In [1]:
# Check if curve_db is loaded, if not load it
try:
    curve_db
    print(f"✓ curve_db is already loaded with {len(curve_db)} isogeny classes")
except NameError:
    print("Loading curve database...")
    curve_db = load('Curve Database (Conductor < 100 000)/curve_database_c1_to_100000.sobj')
    print(f"✓ Loaded curve_db with {len(curve_db)} isogeny classes")
print()

Loading curve database...
✓ Loaded curve_db with 437226 isogeny classes



In [19]:
def elliptic_curve_from_phi(phi):
    """
    Compute the Weierstrass coefficients (a, b) for the elliptic curve:
    y² = x³ + ax + b
    
    where the coefficients are given in terms of φ by equation (70):
    a = -(9φ⁴ - 12φ³ + 30φ² - 12φ + 1) / 3
    b = -2(27φ⁶ - 54φ⁵ - 135φ⁴ + 180φ³ - 99φ² + 18φ - 1) / 27
    
    Parameters:
    -----------
    phi : numeric (int, float, or symbolic)
        The parameter φ
    
    Returns:
    --------
    tuple : (a, b)
        The two Weierstrass coefficients
    """
    # Compute coefficient a
    a_numerator = 9*phi**4 - 12*phi**3 + 30*phi**2 - 12*phi + 1
    a = -a_numerator / 3
    
    # Compute coefficient b
    b_numerator = 27*phi**6 - 54*phi**5 - 135*phi**4 + 180*phi**3 - 99*phi**2 + 18*phi - 1
    b = -2 * b_numerator / 27
    
    return a, b

def elliptic_curve_from_phi_sage(phi):
    """
    SageMath version that returns an actual EllipticCurve object.
    
    Parameters:
    -----------
    phi : numeric or symbolic
        The parameter φ
    
    Returns:
    --------
    EllipticCurve : Sage EllipticCurve object
    """
    from sage.all import EllipticCurve, QQ
    
    a, b = elliptic_curve_from_phi(phi)
    
    # Create elliptic curve in Weierstrass form [0, 0, 0, a, b]
    return EllipticCurve([0, 0, 0, a, b])

In [ ]:
phi = 15/2
elliptic_curve_from_phi_sage(phi).label()

In [7]:
data = load('phi_scan_results/(-500,500)_phi_scan_data')

In [8]:
phis = data['all_phis']

In [10]:
phi_to_iso = data['phi_to_iso']

In [14]:
isos = data['isogeny_classes']

In [23]:
from tqdm import tqdm

phis_class = []

for iso_class in tqdm(isos, desc='Parsing isogeny classes:'):
    phis_tmp = []
    for phi in phis:
        iso = phi_to_iso[phi]
        if iso == iso_class:
            phis_tmp.append(phi)
    phis_class.append(phis_tmp)

Parsing isogeny classes:: 100%|█████████████████████████████████████████████████████| 1296/1296 [01:22<00:00, 15.78it/s]


In [29]:
mults = [len(x) for x in phis_class]

In [37]:
phis_class[6]

[8, 7/71]

In [ ]:
phi_class

In [38]:
def phi_to_isogeny_class(phi):
    """
    Given a parameter φ, compute the corresponding elliptic curve
    and return its isogeny class label.
    
    Parameters:
    -----------
    phi : numeric
        The parameter φ (should result in a curve over Q)
    
    Returns:
    --------
    str : The isogeny class label (e.g., '37a')
    
    Raises:
    -------
    ValueError : If the curve is not defined over Q or has no Cremona label
    """
    from sage.all import EllipticCurve, QQ
    
    # Compute Weierstrass coefficients from phi (equation 70)
    a_numerator = 9*phi**4 - 12*phi**3 + 30*phi**2 - 12*phi + 1
    a = -a_numerator / 3
    
    b_numerator = 27*phi**6 - 54*phi**5 - 135*phi**4 + 180*phi**3 - 99*phi**2 + 18*phi - 1
    b = -2 * b_numerator / 27
    
    # Create elliptic curve
    E = EllipticCurve([0, 0, 0, a, b])
    
    # Ensure we're working over Q
    try:
        E_Q = E.change_ring(QQ)
    except (TypeError, ValueError):
        raise ValueError(f"Curve from φ={phi} is not defined over Q")
    
    # Convert to minimal model
    E_min = E_Q.minimal_model()
    
    # Get the Cremona label
    try:
        label = E_min.label()
    except (RuntimeError, LookupError) as e:
        raise ValueError(f"Could not find Cremona label for curve from φ={phi}: {e}")
    
    # Extract isogeny class from label (e.g., '37a2' -> '37a')
    i = len(label) - 1
    while i >= 0 and label[i].isdigit():
        i -= 1
    isogeny_class = label[:i+1]
    
    return isogeny_class

In [59]:
elliptic_curve_from_phi_sage(phis_class[0][1]).j_invariant()

8805624625/2312

In [16]:
isos_to_phis = load('phi_scan_results/(-500,500)_iso_to_phis')

In [41]:
isos_to_phis

{'34a': [2, 1/17],
 '156b': [3, 1/13],
 '210a': [4, 3/35, 6/5, 1/49, 9/25, -2/7],
 '220a': [5, 1/11],
 '1590u': [6, 5/53],
 '2604f': [7, 3/31],
 '994g': [8, 7/71],
 '30a': [9, 1/25, 1/10, -3/5, 3/2, 1/4],
 '2670e': [10, 9/89],
 '1540c': [11, 5/49],
 '7062t': [12, 11/107],
 '4524e': [13, 3/29],
 '910j': [14, 1/65, 13/125, 8/7],
 '28140s': [15, 7/67],
 '4290bb': [16, 9/169, 20/11, 25/121, 15/143, -12/13],
 '646e': [17, 2/19],
 '16422ba': [18, 17/161],
 '19380i': [19, 9/85],
 '34010h': [20, 19/179],
 '19740v': [21, 5/47],
 '91014q': [22, 21/197],
 '29670w': [24, 23/215],
 '210b': [25, 3/28, 1/16, 9/49, 15/7, -5/4],
 '30290k': [26, 25/233],
 '1716b': [27, 13/121],
 '10542n': [28, 27/251],
 '52780g': [29, 7/65],
 '17794j': [32, 31/287],
 '2442i': [33, 4/37],
 '67830bz': [36, 42/17, 49/289, 35/323, -30/19, 25/361],
 '36852i': [37, 9/83],
 '9430f': [41, 5/46],
 '66660g': [45, 11/101],
 '2310t': [49, -21/11, 14/5, 9/121, 4/25, 6/55],
 '31430l': [50, 49/449],
 '798e': [57, 1/28, 7/64, 27/19],
 

In [64]:
for indx in range(10):
    print([elliptic_curve_from_phi_sage(isos_to_phis[list(isos_to_phis.keys())[indx]][n]).label() for n in range(len(list(isos_to_phis.values())[indx]))])

['34a1', '34a2']
['156b1', '156b2']
['210a1', '210a2', '210a2', '210a4', '210a5', '210a2']
['220a2', '220a1']
['1590u1', '1590u2']
['2604f1', '2604f2']
['994g1', '994g2']
['30a4', '30a5', '30a2', '30a2', '30a2', '30a1']
['2670e1', '2670e2']
['1540c1', '1540c2']
